In [1]:
# Importamos librerías necesarias

import requests
import pandas as pd
import psycopg2
from datetime import datetime

In [2]:
# URL base de la API de CoinCap
base_url = "https://api.coincap.io/v2"


In [3]:
# Endpoint para obtener datos de activos
endpoint = "/assets"

In [4]:
# Encabezados de la solicitud con la clave API
headers = {
    "Accept-Encoding": "gzip, deflate"}

In [5]:
# URL completa para la solicitud
url = base_url + endpoint

In [6]:
# Realizar la solicitud GET a la API
response = requests.get(url, headers=headers)

In [7]:
# Verificar el código de estado de la respuesta
if response.status_code == 200:
    # La solicitud fue exitosa
    data = response.json()
    # Imprimir los datos de los activos (solo imprime los primeros 5 en este ejemplo)
    for asset in data['data'][:5]:
        print(asset)
else:
    # La solicitud falló
    print("Error al realizar la solicitud:", response.status_code)


{'id': 'bitcoin', 'rank': '1', 'symbol': 'BTC', 'name': 'Bitcoin', 'supply': '19685243.0000000000000000', 'maxSupply': '21000000.0000000000000000', 'marketCapUsd': '1189837300157.3691837089125540', 'volumeUsd24Hr': '11217139525.2286809388781974', 'priceUsd': '60443.1096002914052780', 'changePercent24Hr': '-2.6082328009656209', 'vwap24Hr': '63103.4281638439132020', 'explorer': 'https://blockchain.info/'}
{'id': 'ethereum', 'rank': '2', 'symbol': 'ETH', 'name': 'Ethereum', 'supply': '120071733.9252206000000000', 'maxSupply': None, 'marketCapUsd': '354207113978.1147398639114714', 'volumeUsd24Hr': '6223324803.0007848095851188', 'priceUsd': '2949.9625132315584501', 'changePercent24Hr': '-2.6045143096992918', 'vwap24Hr': '3063.5653245051833818', 'explorer': 'https://etherscan.io/'}
{'id': 'tether', 'rank': '3', 'symbol': 'USDT', 'name': 'Tether', 'supply': '108732462510.7544700000000000', 'maxSupply': None, 'marketCapUsd': '108901291006.0586521444639877', 'volumeUsd24Hr': '22366702473.742730

In [8]:
# Convertir el JSON a un DataFrame de pandas
df = pd.DataFrame(data['data'])

In [9]:
#Agregamos columna fecha de consulta
fecha_consulta = datetime.now().timestamp()
df['fecha_consulta'] = pd.to_datetime(fecha_consulta, unit='s')

In [10]:
# Eliminar columnas innecesarias
df.drop(['id'], axis=1, inplace=True)

In [11]:
# Elegimos la columna symbol como ID. Reordenamos y renombramos
symbol_column = df.pop('symbol')  
df.insert(0, 'id', symbol_column)

In [12]:
# Creamos la conexión a Redsshift
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="matiaspereyra_coderhouse"
with open("C:/Users/machi/Desktop/Cursos/Data Engineer/Coder/pwdredshift.txt",'r') as f:
    pwd= f.read()
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [13]:
#Crear cursor
cursor = conn.cursor()

In [14]:
#Crear la tabla
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS matiaspereyra_coderhouse.crypto
        (
	    id VARCHAR(50) primary key  
	    ,rank INTEGER
	    ,name VARCHAR(255)  
	    ,supply DECIMAL
	    ,maxSupply DECIMAL
	    ,marketCapUsd DECIMAL
	    ,volumeUsd24Hr DECIMAL  
	    ,priceUsd DECIMAL 
	    ,changePercent24Hr DECIMAL  
	    ,vwap24Hr DECIMAL
	    ,explorer VARCHAR(255)
	    ,fecha_consulta TIMESTAMP
	    ,fecha_insercion TIMESTAMP
	    )
    """)
    conn.commit()

In [15]:
# Eliminar los registros existentes
cursor.execute("DELETE FROM crypto")
conn.commit()

In [16]:
# Obtener el timestamp de inserción
#fecha_insercion = datetime.now().timestamp()

In [17]:
# Obtener el timestamp de inserción como objeto de fecha y hora
fecha_insercion = datetime.now()

In [18]:
df.head()

,id,rank,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,vwap24Hr,explorer,fecha_consulta
0,BTC,1,Bitcoin,19685243.0000000000000000,21000000.0000000000000000,1189837300157.3691837089125540,11217139525.2286809388781974,60443.1096002914052780,-2.6082328009656209,63103.4281638439132020,https://blockchain.info/,2024-04-17 15:40:42.931407872
1,ETH,2,Ethereum,120071733.9252206000000000,None,354207113978.1147398639114714,6223324803.0007848095851188,2949.9625132315584501,-2.6045143096992918,3063.5653245051833818,https://etherscan.io/,2024-04-17 15:40:42.931407872
2,USDT,3,Tether,108732462510.7544700000000000,None,108901291006.0586521444639877,22366702473.7427305118974532,1.0015526963282698,0.0425996655050704,1.0010149785641852,https://www.omniexplorer.info/asset/31,2024-04-17 15:40:42.931407872
3,BNB,4,BNB,166801148.0000000000000000,166801148.0000000000000000,86957807522.8059890080470576,560044903.2037886138867631,521.3261932873866612,-1.1367368410468888,537.9802524123700615,https://etherscan.io/token/0xB8c77482e45F1F44d...,2024-04-17 15:40:42.931407872
4,SOL,5,Solana,446708850.8686387000000000,None,57530154722.8092168388407203,1657402505.3845136132217524,128.7866909530449487,-0.5551339973190083,135.4809080939909859,https://explorer.solana.com/,2024-04-17 15:40:42.931407872


In [19]:
# Insertamos registros
for index, row in df.iterrows():
    cursor.execute("INSERT INTO crypto (id, rank, name, supply, maxSupply, marketCapUsd, volumeUsd24Hr, priceUsd, changePercent24Hr, vwap24Hr, explorer, fecha_consulta, fecha_insercion) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                   (row['id'], row['rank'], row['name'], row['supply'], row['maxSupply'], row['marketCapUsd'], row['volumeUsd24Hr'], row['priceUsd'], row['changePercent24Hr'], row['vwap24Hr'], row['explorer'], row['fecha_consulta'], fecha_insercion))


# Confirmar la inserción de los registros
conn.commit()



In [20]:
# Consulta
query = "SELECT * FROM crypto"

In [21]:
# Leer la tabla en un DataFrame
df_result = pd.read_sql_query(query, conn)

C:\Users\machi\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [22]:
# Cerrar el cursor y la conexión
cursor.close()
conn.close()

In [23]:
# Mostrar el DataFrame resultante
print(df_result)

      id  rank       name        supply     maxsupply  marketcapusd  \
0    BTC     1    Bitcoin  1.968524e+07  2.100000e+07  1.189837e+12   
1    ETH     2   Ethereum  1.200717e+08           NaN  3.542071e+11   
2   USDT     3     Tether  1.087325e+11           NaN  1.089013e+11   
3    BNB     4        BNB  1.668011e+08  1.668011e+08  8.695781e+10   
4    SOL     5     Solana  4.467089e+08           NaN  5.753015e+10   
..   ...   ...        ...           ...           ...           ...   
95  CELO    96       Celo  5.338178e+08  1.000000e+09  3.982596e+08   
96   FTT    97  FTX Token  3.288951e+08  3.521700e+08  3.972756e+08   
97   NXM    98        NXM  6.594068e+06           NaN  3.969031e+08   
98   ELF    99       aelf  7.263902e+08  1.000000e+09  3.920141e+08   
99  QTUM   100       Qtum  1.047558e+08  1.078224e+08  3.905774e+08   

    volumeusd24hr  priceusd  changepercent24hr  vwap24hr  \
0    1.121714e+10   60443.0               -3.0   63103.0   
1    6.223325e+09    2950.0